# Data modelling

In this Notebook, I detailed the data modelling phase of this project. 

**Star Schema Model**
This project aims to perform analyses on BIXI logs (fact table) and use secondary knowledge linked to these records, such as temperature and day of the week. 

When looking at the data during the exploration phase and based on the analytics goals, I chose to model the data based on the star schema. While it is a simpler model, it fits the purposes of this project perfectly. Indeed, since not all investigations require the same secondary data but constantly require the log, the star schema seems an excellent way to save space and time during analyses. 

![Schema of data modelling](images/data_modeling.png)

## Fact Table

The fact table, which correspond to the core of the star schema, will consist of all the logs from BIXI which includes the following variables:

- **trip_id (PK)** : this will be a unique trip id for each new trips
- **station_start (FK)** : ID of the station where the bike was taken (trips start)
- **station_end (FK)** :  ID of the station where the bike was left (trip end)
- **date_start (FK)** : date-format (yyyy-mm-dd-hh-mm-ss) of the moment the bike ride started
- **date_end (FK)** : date-format of the moment the bike ride ended
- **weather_id (FK)** : date-format (yyyy-mm-dd) ID to the daily weather
- trip_duration : lenght in seconds of the bike ride 
- is_member : if the user has a member pass (1) or not (0)

**Challenge**: My challenge in this section was to find a way to combine the full-length date (yyyy-mm-dd-hh-mm-ss) of the bike with the information of the daily weather (yyyy-mm-dd), which is less granular. 

## Dimension Table

### Stations

This dimension table consist of information about the BIXI stations. Since users move from one station to another and the stations remains the same during a year, it's important that the stations have its own table to reduce the amount of duplicate data. 

I added a "*is_active_YYYY*" variable where one row is added for each year. While BIXI usually place the stations at the same place each year, some might be added to to meet a growing need or to extend its coverage. Thus, for each year, I added a boolean value to see if the station was active that year. 

It as to be noted that BIXI run since 2014 and thus it is reasonable to add the variable like this, but it would definitely be messy if we had many more years. An alternative would be to change the format to a JSON for example. 

- **station_id (PK)** : this is the unique key to identify the station
- latitude : latitude coordonate to the station
- longitude : longitude coordonate to the station
- name : name of the station (intersection of two streets)
- is_active_YYYY : if the station is active that specific year (one row per year)

### Daily Weather

In this table, we can find the daily weather in Montréal. The unique ID for this table is the date (yyyy-mm-dd) since we are suppose to only have one row per day. 

- **weather_id (PK)** : unique id for the daily weather (yyyy-mm-dd)
- temperature : mean temperature of the day (°C)
- precipitation : total precipiation (mm)

### Date

In this last dimension table, I store specific information about the date. 
- **date (PK)** : the date with the format (yyyy-mm-dd-hh-mm)	
	- year (yyyy)
	- month (mm)
	- week (week of the year)
	- day (dd)
	- hour (hh)
	- minute (mm)
- is_weekday: 
	- 1=mon-fri (weekday)
	- 0=sat-sun (weekend)

# Planification of the data Pipeline 

**Using Apache Airflow, I will develop the following data pipeline:**

1. Take all the local raw data and put it in S3 (allows to store lots of data cheaply)
2. Load temporaty the raw data into redshift (staging phase)
3. Transform data in redshift using PostgreSQL into the new tables
    - **Trips** : 
        - Creating a unique ID "trip_id" each time a new row is added
        - Selecting only the desired columns from the BIXI logs file (date_start, date_end, station_start, station_end, trip_duration, is_member)
        - Adding a weather_id based on troncate date from date_start (yyyy-mm-dd)
    - **Stations**:
        - Selecting only the desired columns from the yearly stations file
        - Each year (i.e. new file), add a row for active station. Go through all existing stations: if station exist mark as active, else mark as innactive
    - **Weather**:
        - Selecting only the desired columns from the yearly weather file (temperature, precipitation)
    - **Date**:
        - Troncate each date to get the required section (yyyy, mm, dd, hh ...)
        
4. Data quality checks
    - When adding a new station,we need to make sure that it is located in Montréal, using the following sqaure in x=[-74.0, -73.45] and y=[45.4, 45.7]
    - TBD